In [ ]:
##import Libraries
import os,shutil
from tensorflow import keras
import tensorflow as tf

In [ ]:
##Preproces Data-----> From one directory to categorical directory coping data

path = 'classification'   ##Dataset Whole images in single directory
file_path = 'train_class' ## Destination path where data is categorised using specific class

img_file = os.listdir(path)

for i in sorted(img_file):
    img_class_name = i.split('_')[0] 
    path1 = path + '/' + i
    det_path_dir = file_path +'/'+ img_class_name
    if not os.path.isdir(det_path_dir):
        os.mkdir(det_path_dir)
    shutil.copyfile(path1 ,det_path_dir + '/' + i)

In [ ]:

## Train and Test Data Paths
path_train = 'train_class'    ##Categorised folder for train images
path_test = 'test_class'    ##Categorised folder for test images

In [ ]:
## Create classification model using keras
input_shape = (224, 224,3)
img_input = keras.layers.Input(shape=input_shape)

x = keras.layers.Conv2D(32, (3, 3), activation='relu',padding='same',name='block1_conv1')(img_input)

x = keras.layers.Conv2D(32, (3, 3), activation='relu',padding='same',name='block1_conv2')(x)
x = keras.layers.Dropout(0.2)(x)

x = keras.layers.MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

x = keras.layers.Conv2D(64, (3, 3), activation='relu',padding='same',name='block2_conv1')(x)
x = keras.layers.Dropout(0.2)(x)

x = keras.layers.Conv2D(64, (3, 3), activation='relu',padding='same',name='block2_conv2')(x)
x = keras.layers.Dropout(0.2)(x)

x = keras.layers.MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

x = keras.layers.Conv2D(128, (3, 3), activation='relu',padding='same',name='block3_conv1')(x)
x = keras.layers.Dropout(0.2)(x)

x = keras.layers.MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

x = keras.layers.Flatten(name='flatten')(x)

x = keras.layers.Dense(512, activation='relu', name='fc1')(x)
x = keras.layers.Dropout(0.2)(x)

x = keras.layers.Dense(128, activation='relu', name='fc2')(x)

x = keras.layers.Dense(5, activation='softmax', name='predictions')(x)

model = keras.models.Model(img_input, x)

model.summary()


In [ ]:
## Data Preprocessing and fit on Model
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        path_train,
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary')
validation_generator = test_datagen.flow_from_directory(
        path_test,
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary')


In [ ]:
## Compile and fit model
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size

model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_generator,
        steps_per_epoch=STEP_SIZE_TRAIN,
        epochs=50,verbose =1)

In [ ]:
##Evaluate proposed model with test dataset
model.evaluate_generator(
    validation_generator, steps=STEP_SIZE_VALID)